In [ ]:

import requests
import pandas as pd
import numpy as np
from ta.trend import EMAIndicator
from ta.momentum import RSIIndicator

# ===== 1. Получить пары USDC с Kraken =====
def get_usdc_pairs():
    url = "https://api.kraken.com/0/public/AssetPairs"
    resp = requests.get(url).json()
    pairs = []
    for key, val in resp['result'].items():
        if val.get('wsname') and '/USDC' in val['wsname'] and '.d' not in key:  # Исключаем dark pool
            pairs.append({'pair': key, 'wsname': val['wsname']})
    return pairs

usdc_pairs = get_usdc_pairs()
print(f"Найдено {len(usdc_pairs)} пар к USDC на Kraken")
display(pd.DataFrame(usdc_pairs))

# ===== 2. Кастомная функция SuperTrend =====
def supertrend(df, period=10, multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    atr = df['high'].rolling(window=period).max() - df['low'].rolling(window=period).min()
    atr = atr.rolling(window=period).mean()
    upperband = hl2 + (multiplier * atr)
    lowerband = hl2 - (multiplier * atr)
    st = pd.Series(index=df.index, dtype='float64')
    direction = pd.Series(index=df.index, dtype='int')
    st.iloc[0] = upperband.iloc[0]
    direction.iloc[0] = 1
    for i in range(1, len(df)):
        if df['close'].iloc[i] > upperband.iloc[i-1]:
            direction.iloc[i] = 1
        elif df['close'].iloc[i] < lowerband.iloc[i-1]:
            direction.iloc[i] = -1
        else:
            direction.iloc[i] = direction.iloc[i-1]
            if direction.iloc[i] == 1 and lowerband.iloc[i] > lowerband.iloc[i-1]:
                lowerband.iloc[i] = lowerband.iloc[i-1]
            if direction.iloc[i] == -1 and upperband.iloc[i] < upperband.iloc[i-1]:
                upperband.iloc[i] = upperband.iloc[i-1]
        st.iloc[i] = lowerband.iloc[i] if direction.iloc[i] == 1 else upperband.iloc[i]
    return st

# ===== 3. Функция получения исторических OHLC (15m) =====
def get_ohlc(pair, interval=15):
    url = f"https://api.kraken.com/0/public/OHLC"
    params = {'pair': pair, 'interval': interval}
    resp = requests.get(url, params=params).json()
    key = list(resp['result'].keys())[0]
    ohlc = resp['result'][key]
    df = pd.DataFrame(ohlc, columns=[
        'time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'
    ])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.astype({'open': float, 'high': float, 'low': float, 'close': float, 'volume': float})
    return df

# ===== 4. Проверка условий стратегии =====
def check_strategy(df):
    if len(df) < 80:  # минимум для EMA(75)
        return False
    ema = EMAIndicator(df['close'], window=75).ema_indicator()
    rsi = RSIIndicator(df['close'], window=14).rsi()
    st = supertrend(df, period=10, multiplier=3)
    vol_now = df['volume'].iloc[-1]
    vol_prev = df['volume'].iloc[-2]
    vol_spike = (vol_now - vol_prev) / vol_prev * 100 if vol_prev > 0 else 0

    price = df['close'].iloc[-1]
    # Условия по стратегии:
    if (
        price > ema.iloc[-1] and
        rsi.iloc[-1] < 55 and
        rsi.iloc[-1] > 30 and
        price > st.iloc[-1] and
        vol_spike >= 50
    ):
        return True
    return False

# ===== 5. Основной цикл проверки пар =====
results = []
for p in usdc_pairs:
    print(f"Проверяем {p['wsname']}")
    try:
        df = get_ohlc(p['pair'], interval=15)
        if check_strategy(df):
            results.append(p['wsname'])
            print(f"  >> СИГНАЛ НА ВХОД по {p['wsname']}")
    except Exception as e:
        print(f"Ошибка на {p['wsname']}: {e}")

print("\n=== Пары, подходящие под стратегию прямо сейчас ===")
if results:
    for r in results:
        print(r)
else:
    print("Нет подходящих USDC-пар на данный момент.")
